Implimenting gradient descent using a Value wrapper class 
Followed Andrej Karpathy Follow along

In [159]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt


In [160]:
import math

class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out

    def __truediv__(self, other):
        return self * other**-1

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'{other}')

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad 
        
        out._backward = _backward
    
        return out

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
            
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
    def exp(self):
        n = self.data
        out = Value(math.exp(n), (self,), 'exp')
        
        def _backward():
            self.grad += out.grad * out.data

        out._backward = _backward
        return out


In [161]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [169]:
def mse_loss(pred, target):
    return sum((pi - ti)**2 for pi, ti in zip(pred, target))

In [170]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

xs = [
    [2.0, 3.0, -1.0] ,
    [3.0, -1.0, 0.5] ,
    [0.5, 1.0, 1.0] ,
    [1.0, 1.0, -1.0]
]
ys = [ 1.0, -1.0, 1.0, -1.0]

In [171]:
for k in range(10):
    ypred = [n(x) for x in xs]
    loss = mse_loss(ypred, ys)

    for p in n.parameters():
        p.grad = 0
        
    loss.backward()

    for p in n.parameters():
        p.data += -0.01 * p.grad

    print(k, loss.data)

0 6.311497445504004
1 5.979088027960635
2 5.56045983505836
3 5.050655553118657
4 4.462106430120787
5 3.8233964210831544
6 3.1662140637968705
7 2.5223371307607456
8 1.94552455417447
9 1.5126031891614744
